In [10]:
pwd

'/home/wsuser/work'

In [11]:
!pip install keras==2.2.4
!pip install tensorflow==2.5.0

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen =ImageDataGenerator(rescale=1/255,zoom_range=0.2,horizontal_flip=True,vertical_flip=False)
test_datagen = ImageDataGenerator(rescale=1/255)

In [13]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='5zReQ3KYAVJxnj7DurSBYT4r_mHIpHgTT-rE3-6tidxM',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageclassification-donotdelete-pr-l5gwfenthv2649'
object_key = 'conversation engine for deaf and dumb.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [14]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [15]:
x_train = train_datagen.flow_from_directory('/home/wsuser/work/Dataset/training_set',target_size=(64,64),batch_size=300,class_mode='categorical',color_mode='grayscale')

x_test = test_datagen.flow_from_directory('/home/wsuser/work/Dataset/test_set',target_size=(64,64),batch_size=300,class_mode='categorical',color_mode='grayscale')

Found 15750 images belonging to 9 classes.
Found 2250 images belonging to 9 classes.


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten

In [17]:
model = Sequential()

2022-11-17 16:51:13.229697: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow
2022-11-17 16:51:13.229741: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)


In [18]:
model.add(Convolution2D(32,(3,3),input_shape=(64,64,1),activation='relu'))

In [19]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [20]:
model.add(Flatten())

In [21]:
model.add(Dense(units=512,activation='relu'))
model.add(Dense(units=9,activation='softmax'))

In [22]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [23]:
model.fit_generator(x_train,steps_per_epoch=24,epochs=10,validation_data=x_test,validation_steps=40)

model.save('signlanguage.h5')

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
24/24 [==============================] - 25s 1s/step - loss: 1.0045 - accuracy: 0.6870 - val_loss: 0.3942 - val_accuracy: 0.8960
Epoch 2/10
24/24 [==============================] - 22s 908ms/step - loss: 0.2578 - accuracy: 0.9284
Epoch 3/10
24/24 [==============================] - 22s 910ms/step - loss: 0.1443 - accuracy: 0.9616
Epoch 4/10
24/24 [==============================] - 22s 891ms/step - loss: 0.0910 - accuracy: 0.9780
Epoch 5/10
24/24 [==============================] - 22s 917ms/step - loss: 0.0616 - accuracy: 0.9840
Epoch 6/10
24/24 [==============================] - 22s 924ms/step - loss: 0.0445 - accuracy: 0.9893
Epoch 7/10
24/24 [==============================] - 22s 904ms/step - loss: 0.0371 - accuracy: 0.9918
Epoch 8/10
24/24 [==============================] - 27s 1s/step - loss: 0.0261 - accuracy: 0.9944
Epoch 9/10
24/24 [==============================] - 27s 1s/step - loss: 0.0189 - accuracy: 0.9961
Epoch 10/10
24/24 [==============================] - 23s 9

In [24]:
!tar -zcvf RealtimeModel.tgz signlanguage.h5

signlanguage.h5


In [25]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 18.6 MB/s eta 0:00:01


In [39]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
 "url": "https://us-south.ml.cloud.ibm.com",
 "apikey": "v6C0lUgdmIvM6tZX17jdv7KYb3R2GJaGiX13njJuFIbo"
}
client = APIClient(wml_credentials)

In [40]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return (next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [41]:
space_uid = guid_from_space_name(client, 'RealtimeModel')
print("Space UID : ", space_uid)

Space UID :  0708a8a1-39d8-4752-9639-0eff8325dd66


In [42]:
client.set.default_space(space_uid)

'SUCCESS'

In [43]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [44]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")

In [45]:
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [46]:
model_details = client.repository.store_model(model='RealtimeModel.tgz',
meta_props={
 client.repository.ModelMetaNames.NAME: "CNN",
 client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
 client.repository.ModelMetaNames.TYPE: "tensorflow_2.7"})
model_id = client.repository.get_model_id(model_details)


In [52]:
model_id

'561ce0b2-1cf2-4677-b0fd-9f23987cb9a0'

In [53]:
client.repository.download(model_id,'signlanguage3.tar.gz')

Successfully saved model content to file: 'signlanguage3.tar.gz'


'/home/wsuser/work/signlanguage3.tar.gz'

In [54]:
ls

Dataset/           signlanguage1.tar.gz  signlanguage3.tar.gz
RealtimeModel.tgz  signlanguage2.tar.gz  signlanguage.h5
